In [ ]:
%matplotlib inline
import pandas as pd
from fbprophet import Prophet
import timeit

import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import numpy as np

from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
import io
import pandas as pd
from google.colab import files

files = files.upload()

data = pd.read_excel(io.BytesIO(files.get('UPT.xlsx')))

print (data)

In [ ]:
# Group by NTD ID
ntd = data.groupby('5 digit NTD ID').sum()
ntd = ntd.drop(columns=['UZA'])
ntd

In [ ]:
## NewYork Dataframe

newyork = data
newyork = newyork.groupby('UZA').sum()
newyork = newyork.drop(columns=['5 digit NTD ID'])
newyork = newyork.reset_index().rename(index=str,columns={'index':'ds'})
newyork = newyork[newyork['UZA']==1]
newyork = newyork.T[1:]
newyork = newyork.reset_index().rename(index=str,columns={'index':'ds','1':'y'})


def date_convert(x):
    return datetime.strptime(x,"%b%y")

from datetime import datetime
newyork['ds'] = newyork['ds'].apply(date_convert)

In [ ]:
## Seattle Dataframe

seattle = data
seattle = seattle.groupby('UZA').sum()
seattle = seattle.drop(columns=['5 digit NTD ID'])
seattle = seattle.reset_index().rename(index=str,columns={'index':'ds'})
seattle = seattle[seattle['UZA']==14.0]
seattle = seattle.T[1:]
seattle = seattle.reset_index().rename(index=str,columns={'index':'ds','14':'y'})


def date_convert(x):
    return datetime.strptime(x,"%b%y")

from datetime import datetime
seattle['ds'] = seattle['ds'].apply(date_convert)

In [ ]:
## Washington DC Dataframe

dc = data
dc = dc.groupby('UZA').sum()
dc = dc.drop(columns=['5 digit NTD ID'])
dc = dc.reset_index().rename(index=str,columns={'index':'ds'})
dc = dc[dc['UZA']==14.0]
dc = dc.T[1:]
dc = dc.reset_index().rename(index=str,columns={'index':'ds','8':'y'})


def date_convert(x):
    return datetime.strptime(x,"%b%y")

from datetime import datetime
dc['ds'] = dc['ds'].apply(date_convert)

In [ ]:
## Los Angeles Dataframe

la = data
la = la.groupby('UZA').sum()
la = la.drop(columns=['5 digit NTD ID'])
la = la.reset_index().rename(index=str,columns={'index':'ds'})
la = la[la['UZA']==2]
la = la.T[1:]
la = la.reset_index().rename(index=str,columns={'index':'ds','2':'y'})


def date_convert(x):
    return datetime.strptime(x,"%b%y")

from datetime import datetime
la['ds'] = la['ds'].apply(date_convert)

In [ ]:
## SanFrancisco Dataframe

sfo = data
sfo = sfo.groupby('UZA').sum()
sfo = sfo.drop(columns=['5 digit NTD ID'])
sfo = sfo.reset_index().rename(index=str,columns={'index':'ds'})
sfo = sfo[sfo['UZA']==2]
sfo = sfo.T[1:]
sfo = sfo.reset_index().rename(index=str,columns={'index':'ds','13':'y'})


def date_convert(x):
    return datetime.strptime(x,"%b%y")

from datetime import datetime
sfo['ds'] = sfo['ds'].apply(date_convert)

In [ ]:
## Master Dataframe
master = ntd.T
master = master.aggregate(func='sum',axis=1)
master = master.reset_index().rename(index=str,columns={'index':'ds',0:'y'})
master

def date_convert(x):
    return datetime.strptime(x,"%b%y")

from datetime import datetime
master['ds'] = master['ds'].apply(date_convert)
master

In [ ]:
precovid = master[['ds','y']][:-26]
precovid

In [ ]:
## Master trips plot
ax = master.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('public transit trips')
ax.set_xlabel('Date')
plt.show()

In [ ]:
# autocorrelation factor
plot_acf(precovid['trips'], ax=plt.gca())
plt.show()

# partial autocorrelation factor
plot_pacf(precovid['trips'], ax=plt.gca())
plt.show

In [ ]:
result = adfuller(precovid['trips'], autolag='AIC')

print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

In [ ]:
## New York trips Plot
newyork_precovid = newyork[['ds','y']][:-26]
newyork_precovid

ax = newyork.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('newyork-Newark trips')
ax.set_xlabel('Date')

plt.show()

In [ ]:
## Seattle trips Plot
seattle_precovid = seattle[['ds','y']][:-26]
seattle_precovid

ax = seattle.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('seattlettle trips')
ax.set_xlabel('Date')

plt.show()

In [ ]:
## Washington DC trips plot
dc_precovid = dc[['ds','y']][:-26]
dc_precovid

ax = dc.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('dclly trips')
ax.set_xlabel('Date')

plt.show()

In [ ]:
## Los Angeles trips plot
la_precovid = la[['ds','y']][:-26]
la_precovid

ax = la.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('la trips')
ax.set_xlabel('Date')

plt.show()

In [ ]:
## SanFrancisco trips plot
sfo_precovid = sfo[['ds','y']][:-26]
sfo_precovid

ax = sfo.set_index('ds').plot(figsize=(12, 8))
ax.set_ylabel('SanFrancisco trips')
ax.set_xlabel('Date')

plt.show()

In [ ]:
## master Prophet Model
master_model = Prophet(interval_width=.95)
master_model.fit(precovid)

## Build out future dates for predictions
future_dates = master_model.make_future_dataframe(periods=128, freq='W')

# Use model to generate forecasts for future dates.
forecast = master_model.predict(future_dates)
print(forecast[['ds', 'trend','yhat', 'yhat_lower', 'yhat_upper']][-20:])
master_model.plot(forecast,uncertainty=True)
plt.show()

plt.figure(figsize=(12,8))
plt.plot(forecast['ds'],forecast['yhat'],color='tab:red')
plt.plot(master['ds'],master['y'],color = 'b')
plt.title('USA')
plt.ylabel('Public Transportation Trips')
plt.legend(['Predicted Ridership','Actual Ridership'])
plt.show()

In [ ]:
## New York Prophet Model
newyork_model = Prophet(interval_width=.95)
newyork_model.fit(newyork_precovid)

## Build out future dates for predictions
future_dates = newyork_model.make_future_dataframe(periods=128, freq='W')

# Use model to generate forecasts for future dates.
forecast = newyork_model.predict(future_dates)
print(forecast[['ds', 'trend','yhat', 'yhat_lower', 'yhat_upper']][-20:])
newyork_model.plot(forecast,uncertainty=True)
plt.show()

plt.figure(figsize=(12,8))
plt.plot(forecast['ds'],forecast['yhat'],color='tab:red')
plt.plot(newyork['ds'],newyork['y'],color = 'b')
plt.title('New York')
plt.ylabel('Public Transportation Trips')
plt.legend(['Predicted Ridership','Actual Ridership'])
plt.show()

In [ ]:
## Seattle Prophet Model
seattle_model = Prophet(interval_width=.95)
seattle_model.fit(seattle_precovid)

## Build out future dates for predictions
future_dates = seattle_model.make_future_dataframe(periods=128, freq='W')

# Use model to generate forecasts for future dates.
forecast = seattle_model.predict(future_dates)
print(forecast[['ds', 'trend','yhat', 'yhat_lower', 'yhat_upper']][-20:])
seattle_model.plot(forecast,uncertainty=True)
plt.show()

plt.figure(figsize=(12,8))
plt.plot(forecast['ds'],forecast['yhat'],color='tab:red')
plt.plot(seattle['ds'],seattle['y'],color = 'b')
plt.title('seattle')
plt.ylabel('Public Transportation Trips')
plt.legend(['Predicted Ridership','Actual Ridership'])
plt.show()

In [ ]:
## Washington DC Prophet Model
dc_model = Prophet(interval_width=.95)
dc_model.fit(dc_precovid)

## Build out future dates for predictions
future_dates = dc_model.make_future_dataframe(periods=128, freq='W')

# Use model to generate forecasts for future dates.
forecast = dc_model.predict(future_dates)
print(forecast[['ds', 'trend','yhat', 'yhat_lower', 'yhat_upper']][-20:])
dc_model.plot(forecast,uncertainty=True)
plt.show()

plt.figure(figsize=(12,8))
plt.plot(forecast['ds'],forecast['yhat'],color='tab:red')
plt.plot(dc['ds'],dc['y'],color = 'b')
plt.title('washington dc')
plt.ylabel('Public Transportation Trips')
plt.legend(['Predicted Ridership','Actual Ridership'])
plt.show()

In [ ]:
## Los Angeles Prophet Model
la_model = Prophet(interval_width=.95)
la_model.fit(la_precovid)

## Build out future dates for predictions
future_dates = la_model.make_future_dataframe(periods=128, freq='W')

# Use model to generate forecasts for future dates.
forecast = la_model.predict(future_dates)
print(forecast[['ds', 'trend','yhat', 'yhat_lower', 'yhat_upper']][-20:])
la_model.plot(forecast,uncertainty=True)
plt.show()

plt.figure(figsize=(10,6))
plt.plot(forecast['ds'],forecast['yhat'],color='tab:red')
plt.plot(la['ds'],la['y'],color = 'b')
plt.title('Los Angeles')
plt.ylabel('Public Transportation Trips')
plt.legend(['Predicted Ridership','Actual Ridership'])
plt.show()

In [ ]:
## SanFrancisco Prophet Model
sfo_model = Prophet(interval_width=.95)
sfo_model.fit(sfo_precovid)

## Build out future dates for predictions
future_dates = sfo_model.make_future_dataframe(periods=128, freq='W')

# Use model to generate forecasts for future dates.
forecast = sfo_model.predict(future_dates)
print(forecast[['ds', 'trend','yhat', 'yhat_lower', 'yhat_upper']][-20:])
sfo_model.plot(forecast,uncertainty=True)
plt.show()

plt.figure(figsize=(10,6))
plt.plot(forecast['ds'],forecast['yhat'],color='tab:red')
plt.plot(sfo['ds'],sfo['y'],color = 'b')
plt.title('SanFrancisco')
plt.ylabel('Public Transportation Trips')
plt.legend(['Predicted Ridership','Actual Ridership'])
plt.show()